# Data Dictionary
Row: 63326 rows

Columns: 11 columns
- Agency: 16 types 'XXX'
- Agency Type: Airlines/Travel Agency
- Distribution Channel: Online/Offline
- Product Name: 26 types
- Claim: Yes/No
- Duration: -2 to 4881
- Destination: 
- Net Sale: -389 to 810
- Commision: 0 to 283.5
- Gender: F/M
- Age: 0 to 118

In [ ]:
# Import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

In [ ]:
# Select colour pastel
colors = sns.color_palette('pastel')
colors

In [ ]:
# Format code to pep8 standard
# Type in terminal: autopep8 --in-place -a -a Testing01.ipynb

In [ ]:
# Read data
df = pd.read_csv('Dataset/Travel Insurance.csv')

In [ ]:
# Display the column name
df.columns 

In [ ]:
# First five rows of the data
df.head()

In [ ]:
# The T is inverting the position of the columns name and the statistics column
df.describe().T

In [20]:
# The count and data type of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63326 entries, 0 to 63325
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Agency                63326 non-null  object 
 1   Agency Type           63326 non-null  object 
 2   Distribution Channel  63326 non-null  object 
 3   Product Name          63326 non-null  object 
 4   Claim                 63326 non-null  object 
 5   Duration              63326 non-null  int64  
 6   Destination           63326 non-null  object 
 7   Net Sales             63326 non-null  float64
 8   Commision (in value)  63326 non-null  float64
 9   Gender                18219 non-null  object 
 10  Age                   63326 non-null  int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 5.3+ MB
